# 🧠 Advanced LLM Training & Fine-Tuning Guide

## From Pre-trained Models to Custom AI Assistants

This notebook covers **everything you need to know** about training and fine-tuning Large Language Models (LLMs):

### What You'll Learn:
1. **LLM Architecture** - Transformers, Attention, Tokenization
2. **Dataset Preparation** - Creating training data for LLMs
3. **Fine-Tuning Strategies** - Full fine-tuning vs Parameter-Efficient methods
4. **LoRA & QLoRA** - Train billion-parameter models on consumer GPUs
5. **Training Loop** - Custom training with gradient accumulation
6. **Evaluation** - Perplexity, BLEU, ROUGE metrics
7. **Inference Optimization** - Quantization and deployment

---

## 🎯 Understanding LLMs

### What is an LLM?
A **Large Language Model** is a neural network trained on massive text data to:
- Predict the next token in a sequence
- Understand and generate human language
- Follow instructions and complete tasks

### Key Architecture: Transformers

```
Input: "The cat sat on the"
         ↓
    [Tokenization]
         ↓
    [Embedding Layer] → Convert tokens to vectors
         ↓
    [Transformer Blocks] × N
        ├── Multi-Head Self-Attention
        ├── Feed-Forward Network
        └── Layer Normalization
         ↓
    [Output Layer] → Probability distribution over vocabulary
         ↓
Output: "mat" (highest probability next token)
```

## 📦 Section 1: Install Required Libraries

We'll use the **Hugging Face** ecosystem - the industry standard for LLM work:
- `transformers` - Pre-trained models and training utilities
- `datasets` - Efficient data loading and processing
- `peft` - Parameter-Efficient Fine-Tuning (LoRA, etc.)
- `bitsandbytes` - Quantization for memory efficiency
- `accelerate` - Distributed training and mixed precision

In [ ]:
# Install required packages (uncomment to run)
# !pip install transformers datasets accelerate peft bitsandbytes trl sentencepiece
# !pip install torch torchvision torchaudio  # If not already installed

# Core imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️ No GPU detected. Training will be slow on CPU.")
    print("   Consider using Google Colab or a cloud GPU.")

In [ ]:
# Hugging Face imports
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup,
    pipeline
)
from datasets import load_dataset, Dataset as HFDataset
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)

print("✅ All libraries imported successfully!")
print("\n📚 Key Libraries:")
print("   - transformers: For loading and training models")
print("   - datasets: For efficient data handling")
print("   - peft: For LoRA and parameter-efficient training")
print("   - bitsandbytes: For quantization (4-bit, 8-bit)")

---

## 🔤 Section 2: Understanding Tokenization

### Why Tokenization?
LLMs don't see text - they see **numbers**. Tokenization converts text to token IDs.

### Types of Tokenization:
| Method | Example | Pros | Cons |
|--------|---------|------|------|
| **Word-level** | "Hello world" → [1, 2] | Simple | Huge vocabulary, OOV issues |
| **Character-level** | "Hi" → [8, 9] | Small vocab | Long sequences |
| **Subword (BPE)** | "playing" → ["play", "ing"] | Balanced | Requires training |

Modern LLMs use **Byte-Pair Encoding (BPE)** or **SentencePiece**.

In [ ]:
# Load a tokenizer from a pre-trained model
model_name = "gpt2"  # Small model for demonstration
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 doesn't have a pad token by default
tokenizer.pad_token = tokenizer.eos_token

print("🔤 TOKENIZATION DEEP DIVE")
print("="*60)
print(f"Model: {model_name}")
print(f"Vocabulary size: {tokenizer.vocab_size:,}")
print(f"Max sequence length: {tokenizer.model_max_length:,}")

# Example tokenization
text = "Machine learning is transforming the world of artificial intelligence!"

# Tokenize
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.encode(text)

print(f"\n📝 Original text: '{text}'")
print(f"\n🔢 Tokens ({len(tokens)}): {tokens}")
print(f"\n🆔 Token IDs: {token_ids}")

# Decode back
decoded = tokenizer.decode(token_ids)
print(f"\n🔄 Decoded: '{decoded}'")

In [ ]:
# Visualize tokenization
def visualize_tokens(text, tokenizer):
    """Visualize how text is split into tokens"""
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    
    print("Token Breakdown:")
    print("-" * 50)
    for token, tid in zip(tokens, token_ids):
        # Clean token representation (Ġ in GPT-2 means space before)
        clean_token = token.replace('Ġ', '▁')  # Show space marker
        print(f"  '{clean_token:15}' → ID: {tid:6}")

# Examples
examples = [
    "Hello, how are you?",
    "The quick brown fox jumps",
    "TensorFlow and PyTorch are frameworks",
    "I'm learning about LLMs!",
    "🚀 Emojis work too! 🎉"
]

for text in examples:
    print(f"\n📝 Text: '{text}'")
    visualize_tokens(text, tokenizer)
    print()

---

## 🏗️ Section 3: Building a Transformer from Scratch

Let's build a **mini GPT** to understand the architecture deeply!

### Self-Attention Mechanism

The core of transformers is **self-attention**:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where:
- $Q$ = Query (what am I looking for?)
- $K$ = Key (what do I contain?)
- $V$ = Value (what information do I have?)
- $d_k$ = dimension of keys (scaling factor)

In [ ]:
# Build a Mini GPT from scratch!

class SelfAttention(nn.Module):
    """
    Multi-Head Self-Attention mechanism.
    This is the CORE of all transformer models!
    """
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        
        # Linear projections for Q, K, V
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.scale = self.head_dim ** -0.5  # 1/sqrt(d_k)
    
    def forward(self, x, mask=None):
        batch_size, seq_len, embed_dim = x.shape
        
        # Project to Q, K, V
        Q = self.q_proj(x)  # (batch, seq, embed)
        K = self.k_proj(x)
        V = self.v_proj(x)
        
        # Reshape for multi-head attention
        # (batch, seq, embed) -> (batch, num_heads, seq, head_dim)
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Compute attention scores: (Q @ K^T) / sqrt(d_k)
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        
        # Apply causal mask (for autoregressive generation)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))
        
        # Softmax and dropout
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Apply attention to values
        output = torch.matmul(attn_weights, V)
        
        # Reshape back: (batch, num_heads, seq, head_dim) -> (batch, seq, embed)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, embed_dim)
        
        return self.out_proj(output)

print("✅ Self-Attention module created!")
print("\n💡 Key insight: Attention allows each token to 'look at' all other tokens")
print("   and decide which ones are most relevant for prediction.")

In [ ]:
class TransformerBlock(nn.Module):
    """
    A single Transformer block: Attention + Feed-Forward + Residuals + LayerNorm
    """
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        
        # Layer normalization (Pre-LN architecture, used in GPT-2+)
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)
        
        # Self-attention
        self.attention = SelfAttention(embed_dim, num_heads, dropout)
        
        # Feed-forward network (MLP)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),  # GPT uses GELU activation
            nn.Linear(ff_dim, embed_dim),
            nn.Dropout(dropout)
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Pre-norm + Attention + Residual
        x = x + self.dropout(self.attention(self.ln1(x), mask))
        
        # Pre-norm + FFN + Residual
        x = x + self.dropout(self.ffn(self.ln2(x)))
        
        return x


class MiniGPT(nn.Module):
    """
    A minimal GPT-style language model.
    This is the architecture used by GPT-2, GPT-3, and most modern LLMs!
    """
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, max_seq_len, ff_dim=None, dropout=0.1):
        super().__init__()
        
        self.embed_dim = embed_dim
        ff_dim = ff_dim or 4 * embed_dim  # GPT uses 4x expansion
        
        # Token and position embeddings
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        
        # Stack of transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])
        
        # Final layer norm and output projection
        self.ln_final = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
        
        # Weight tying (optional but common)
        self.token_embedding.weight = self.lm_head.weight
        
        self.dropout = nn.Dropout(dropout)
        self.max_seq_len = max_seq_len
        
        # Create causal mask
        self.register_buffer(
            "causal_mask",
            torch.tril(torch.ones(max_seq_len, max_seq_len)).view(1, 1, max_seq_len, max_seq_len)
        )
    
    def forward(self, input_ids, labels=None):
        batch_size, seq_len = input_ids.shape
        
        # Get embeddings
        positions = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = self.dropout(x)
        
        # Get causal mask for current sequence length
        mask = self.causal_mask[:, :, :seq_len, :seq_len]
        
        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x, mask)
        
        # Final layer norm and projection to vocabulary
        x = self.ln_final(x)
        logits = self.lm_head(x)  # (batch, seq, vocab_size)
        
        # Calculate loss if labels provided
        loss = None
        if labels is not None:
            # Shift logits and labels for next-token prediction
            shift_logits = logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )
        
        return {"logits": logits, "loss": loss}

print("✅ MiniGPT architecture created!")
print("\n📊 Architecture Summary:")
print("   1. Token Embedding: Convert token IDs to vectors")
print("   2. Position Embedding: Add positional information")
print("   3. N × Transformer Blocks: Attention + FFN")
print("   4. LM Head: Project back to vocabulary for prediction")

In [ ]:
# Test our Mini GPT
config = {
    "vocab_size": tokenizer.vocab_size,
    "embed_dim": 256,
    "num_heads": 8,
    "num_layers": 4,
    "max_seq_len": 512,
    "dropout": 0.1
}

model = MiniGPT(**config).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("🤖 MINI GPT MODEL")
print("="*60)
print(f"Configuration:")
for k, v in config.items():
    print(f"   {k}: {v}")
print(f"\n📊 Parameters:")
print(f"   Total: {total_params:,} ({total_params/1e6:.2f}M)")
print(f"   Trainable: {trainable_params:,}")

# Test forward pass
test_input = torch.randint(0, config["vocab_size"], (2, 64)).to(device)
with torch.no_grad():
    output = model(test_input)
print(f"\n✅ Forward pass successful!")
print(f"   Input shape: {test_input.shape}")
print(f"   Output logits shape: {output['logits'].shape}")

---

## 📚 Section 4: Dataset Preparation for LLM Training

### Types of Training Data:

| Format | Use Case | Example |
|--------|----------|---------|
| **Raw Text** | Pre-training | Books, Wikipedia, Web pages |
| **Instruction-Following** | Fine-tuning | `{"instruction": ..., "response": ...}` |
| **Conversation** | Chat models | `[{"role": "user"}, {"role": "assistant"}]` |
| **Preference** | RLHF/DPO | `{"chosen": ..., "rejected": ...}` |

In [ ]:
# Load a sample dataset from Hugging Face
print("📚 LOADING TRAINING DATASET")
print("="*60)

# Load a small instruction-following dataset
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")  # First 1000 examples

print(f"Dataset size: {len(dataset)}")
print(f"Features: {dataset.features}")
print(f"\n📝 Sample entry:")
sample = dataset[0]
for key, value in sample.items():
    if isinstance(value, str) and len(value) > 200:
        print(f"   {key}: {value[:200]}...")
    else:
        print(f"   {key}: {value}")

In [ ]:
# Create instruction prompt templates
def format_instruction(example):
    """
    Format instruction-following data into a prompt.
    This is how we teach the model to follow instructions!
    """
    instruction = example["instruction"]
    input_text = example.get("input", "")
    output = example["output"]
    
    if input_text:
        prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        prompt = f"""### Instruction:
{instruction}

### Response:
{output}"""
    
    return {"text": prompt}

# Apply formatting
formatted_dataset = dataset.map(format_instruction)

print("📝 FORMATTED PROMPT EXAMPLE:")
print("="*60)
print(formatted_dataset[0]["text"][:500])
print("...")

In [ ]:
# Tokenize the dataset for training
def tokenize_function(examples):
    """Tokenize texts and create labels for causal LM"""
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None  # Return lists for dataset mapping
    )
    # For causal LM, labels = input_ids (shifted internally)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset.column_names
)

# Split into train/validation
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print("🔢 TOKENIZED DATASET")
print("="*60)
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")
print(f"Features: {train_dataset.features}")
print(f"\nSample tokenized entry:")
print(f"   input_ids length: {len(train_dataset[0]['input_ids'])}")
print(f"   First 10 tokens: {train_dataset[0]['input_ids'][:10]}")

---

## 🎯 Section 5: Fine-Tuning Strategies

### Full Fine-Tuning vs Parameter-Efficient Fine-Tuning

| Method | Parameters Updated | Memory | Speed | Best For |
|--------|-------------------|--------|-------|----------|
| **Full Fine-Tuning** | All | Very High | Slow | Small models, large datasets |
| **LoRA** | ~0.1-1% | Low | Fast | Most cases |
| **QLoRA** | ~0.1-1% + Quantization | Very Low | Medium | Limited GPU memory |
| **Prefix Tuning** | Virtual tokens | Low | Fast | Specific tasks |

### LoRA (Low-Rank Adaptation)

Instead of updating all weights $W$, we learn low-rank matrices $A$ and $B$:

$$W_{new} = W_{frozen} + \alpha \cdot BA$$

Where:
- $W \in \mathbb{R}^{d \times k}$ (original weight)
- $B \in \mathbb{R}^{d \times r}$ (down projection)
- $A \in \mathbb{R}^{r \times k}$ (up projection)
- $r \ll \min(d, k)$ (rank, typically 8-64)

In [ ]:
# Load a pre-trained model for fine-tuning
print("🤖 LOADING PRE-TRAINED MODEL")
print("="*60)

# Using GPT-2 small for demonstration (can scale to larger models)
model_name = "gpt2"

# Load with optional quantization for memory efficiency
use_quantization = False  # Set to True if you have limited GPU memory

if use_quantization and torch.cuda.is_available():
    # 4-bit quantization config (QLoRA)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )
    print("✅ Model loaded with 4-bit quantization (QLoRA)")
else:
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    base_model = base_model.to(device)
    print("✅ Model loaded in full precision")

# Model info
total_params = sum(p.numel() for p in base_model.parameters())
print(f"\n📊 Model: {model_name}")
print(f"   Total parameters: {total_params:,} ({total_params/1e6:.2f}M)")
print(f"   Model size: ~{total_params * 4 / 1e9:.2f} GB (FP32)")

In [ ]:
# Apply LoRA for efficient fine-tuning
print("🔧 APPLYING LoRA")
print("="*60)

# LoRA configuration
lora_config = LoraConfig(
    r=16,                        # Rank of the update matrices
    lora_alpha=32,               # Scaling factor
    target_modules=["c_attn", "c_proj"],  # Which layers to apply LoRA to
    lora_dropout=0.05,           # Dropout for LoRA layers
    bias="none",                 # Don't train bias terms
    task_type=TaskType.CAUSAL_LM # Task type
)

# Prepare model for LoRA
if use_quantization:
    base_model = prepare_model_for_kbit_training(base_model)

# Apply LoRA
model_lora = get_peft_model(base_model, lora_config)

# Compare parameters
trainable_params = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_lora.parameters())

print(f"\n📊 LoRA Statistics:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Trainable %: {100 * trainable_params / total_params:.4f}%")
print(f"\n💡 We're only training {trainable_params/1e6:.2f}M parameters instead of {total_params/1e6:.2f}M!")
print("   This makes fine-tuning possible on consumer GPUs!")

# Show LoRA modules
print("\n🎯 LoRA applied to these modules:")
model_lora.print_trainable_parameters()

---

## 🔄 Section 6: The Training Loop - Deep Dive

Let's build a **complete training loop** from scratch to understand every step!

In [ ]:
# Custom Training Loop for LLMs
class LLMTrainer:
    """
    A complete training loop for LLMs.
    Shows exactly what happens during training!
    """
    def __init__(
        self, 
        model, 
        tokenizer,
        train_dataset,
        eval_dataset=None,
        batch_size=4,
        learning_rate=2e-5,
        num_epochs=3,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_grad_norm=1.0,
        device="cuda"
    ):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
        self.batch_size = batch_size
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.max_grad_norm = max_grad_norm
        self.num_epochs = num_epochs
        
        # Data collator for language modeling
        self.data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False  # Causal LM, not Masked LM
        )
        
        # Create dataloaders
        self.train_dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            collate_fn=self.data_collator
        )
        
        self.eval_dataloader = None
        if eval_dataset:
            self.eval_dataloader = DataLoader(
                eval_dataset,
                batch_size=batch_size,
                collate_fn=self.data_collator
            )
        
        # Calculate total steps
        total_steps = len(self.train_dataloader) * num_epochs // gradient_accumulation_steps
        
        # Optimizer - AdamW is standard for transformers
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=learning_rate,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=0.01
        )
        
        # Learning rate scheduler with warmup
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps
        )
        
        # Training history
        self.history = {
            "train_loss": [],
            "eval_loss": [],
            "learning_rate": []
        }
    
    def train(self):
        """Main training loop"""
        print("🚀 STARTING TRAINING")
        print("="*60)
        print(f"   Epochs: {self.num_epochs}")
        print(f"   Batch size: {self.batch_size}")
        print(f"   Gradient accumulation: {self.gradient_accumulation_steps}")
        print(f"   Effective batch size: {self.batch_size * self.gradient_accumulation_steps}")
        print("="*60)
        
        global_step = 0
        
        for epoch in range(self.num_epochs):
            self.model.train()
            epoch_loss = 0
            progress_bar = tqdm(self.train_dataloader, desc=f"Epoch {epoch+1}/{self.num_epochs}")
            
            for step, batch in enumerate(progress_bar):
                # Move batch to device
                batch = {k: v.to(self.device) for k, v in batch.items()}
                
                # Forward pass
                outputs = self.model(**batch)
                loss = outputs.loss
                
                # Scale loss for gradient accumulation
                loss = loss / self.gradient_accumulation_steps
                
                # Backward pass
                loss.backward()
                
                epoch_loss += loss.item() * self.gradient_accumulation_steps
                
                # Update weights every gradient_accumulation_steps
                if (step + 1) % self.gradient_accumulation_steps == 0:
                    # Gradient clipping (prevents exploding gradients)
                    torch.nn.utils.clip_grad_norm_(
                        self.model.parameters(), 
                        self.max_grad_norm
                    )
                    
                    # Optimizer step
                    self.optimizer.step()
                    self.scheduler.step()
                    self.optimizer.zero_grad()
                    
                    global_step += 1
                    
                    # Log learning rate
                    current_lr = self.scheduler.get_last_lr()[0]
                    self.history["learning_rate"].append(current_lr)
                
                # Update progress bar
                progress_bar.set_postfix({
                    "loss": f"{loss.item() * self.gradient_accumulation_steps:.4f}",
                    "lr": f"{self.scheduler.get_last_lr()[0]:.2e}"
                })
            
            # Epoch metrics
            avg_train_loss = epoch_loss / len(self.train_dataloader)
            self.history["train_loss"].append(avg_train_loss)
            
            print(f"\n📊 Epoch {epoch+1} Summary:")
            print(f"   Train Loss: {avg_train_loss:.4f}")
            
            # Evaluation
            if self.eval_dataloader:
                eval_loss = self.evaluate()
                self.history["eval_loss"].append(eval_loss)
                print(f"   Eval Loss: {eval_loss:.4f}")
                print(f"   Perplexity: {np.exp(eval_loss):.2f}")
        
        return self.history
    
    def evaluate(self):
        """Evaluation loop"""
        self.model.eval()
        total_loss = 0
        
        with torch.no_grad():
            for batch in self.eval_dataloader:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                outputs = self.model(**batch)
                total_loss += outputs.loss.item()
        
        return total_loss / len(self.eval_dataloader)

print("✅ LLMTrainer class created!")
print("\n📋 Key Training Concepts:")
print("   1. Gradient Accumulation: Simulate larger batches with limited memory")
print("   2. Warmup: Gradually increase learning rate at start")
print("   3. Gradient Clipping: Prevent exploding gradients")
print("   4. AdamW: Adam optimizer with proper weight decay")

In [ ]:
# Train the model (using Hugging Face Trainer for convenience)
print("🏋️ TRAINING WITH HUGGING FACE TRAINER")
print("="*60)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llm_output",
    num_train_epochs=1,                    # Reduce for demo
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,         # Effective batch = 2 * 4 = 8
    warmup_steps=50,
    learning_rate=2e-4,                    # Higher LR for LoRA
    fp16=torch.cuda.is_available(),        # Mixed precision if GPU
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",                      # Disable wandb/tensorboard
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    gradient_checkpointing=False,          # Trade compute for memory
)

# Create trainer
trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

print("📋 Training Configuration:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Mixed precision (FP16): {training_args.fp16}")
print("\n⏳ Training... (this may take a few minutes)")

In [ ]:
# Run training (uncomment to actually train)
# train_result = trainer.train()

# For demo purposes, let's show what the training output looks like
print("📊 EXAMPLE TRAINING OUTPUT:")
print("-"*60)
print("""
{'loss': 3.2451, 'learning_rate': 0.0002, 'epoch': 0.05}
{'loss': 2.8923, 'learning_rate': 0.00019, 'epoch': 0.10}
{'loss': 2.5674, 'learning_rate': 0.00018, 'epoch': 0.15}
...
{'eval_loss': 2.1234, 'eval_runtime': 5.23, 'epoch': 1.0}
""")

print("\n💡 Training Tips:")
print("   1. Watch for loss decreasing over time")
print("   2. Eval loss should track train loss (not diverge)")
print("   3. If eval loss increases = overfitting!")
print("   4. Learning rate should decrease (cosine schedule)")

---

## 📏 Section 7: Evaluation Metrics for LLMs

### Key Metrics:

| Metric | Formula/Description | Use Case |
|--------|---------------------|----------|
| **Perplexity** | $PPL = e^{-\frac{1}{N}\sum \log P(x_i)}$ | Language modeling quality |
| **BLEU** | n-gram precision with brevity penalty | Translation, generation |
| **ROUGE** | Recall-oriented n-gram overlap | Summarization |
| **F1 Score** | Harmonic mean of P & R | Classification tasks |
| **Exact Match** | Percentage of perfect matches | QA tasks |

### Perplexity Explained

Perplexity measures how "surprised" the model is by the text:
- **Lower = Better** (model predicts well)
- A perplexity of 10 means the model is as uncertain as randomly choosing from 10 equally likely options

In [ ]:
# Calculate Perplexity
def calculate_perplexity(model, dataloader, device):
    """
    Calculate perplexity on a dataset.
    Perplexity = exp(average cross-entropy loss)
    """
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating perplexity"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            
            # Get loss (averaged over tokens)
            loss = outputs.loss.item()
            
            # Count non-padding tokens
            num_tokens = (batch["attention_mask"].sum()).item()
            
            total_loss += loss * num_tokens
            total_tokens += num_tokens
    
    avg_loss = total_loss / total_tokens
    perplexity = np.exp(avg_loss)
    
    return perplexity, avg_loss

# Example perplexity values for reference
print("📊 PERPLEXITY BENCHMARKS")
print("="*60)
print("""
Model Performance Examples:
┌─────────────────────────┬─────────────┬─────────────────────┐
│ Model                   │ Perplexity  │ Quality             │
├─────────────────────────┼─────────────┼─────────────────────┤
│ Random baseline         │ ~50,000     │ Completely random   │
│ N-gram model            │ ~200-500    │ Basic patterns      │
│ GPT-2 Small (117M)      │ ~35-40      │ Good fluency        │
│ GPT-2 Large (774M)      │ ~20-25      │ Very good           │
│ GPT-3 (175B)            │ ~15-20      │ Excellent           │
│ Fine-tuned on domain    │ ~5-15       │ Domain expert       │
└─────────────────────────┴─────────────┴─────────────────────┘

💡 Lower perplexity = better language modeling
   But doesn't always mean better task performance!
""")

---

## 🎭 Section 8: Text Generation & Inference

Now let's generate text with our model!

In [ ]:
# Text Generation Strategies
print("🎭 TEXT GENERATION STRATEGIES")
print("="*60)

def generate_text(model, tokenizer, prompt, max_length=100, **kwargs):
    """Generate text with various decoding strategies"""
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            **kwargs
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Load base GPT-2 for generation demo
gen_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

prompt = "The future of artificial intelligence"

# Different generation strategies
strategies = {
    "Greedy (deterministic)": {
        "do_sample": False
    },
    "Temperature Sampling (creative)": {
        "do_sample": True,
        "temperature": 0.8,
        "top_k": 0
    },
    "Top-K Sampling": {
        "do_sample": True,
        "top_k": 50,
        "temperature": 0.7
    },
    "Top-P (Nucleus) Sampling": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.7
    },
    "Beam Search": {
        "do_sample": False,
        "num_beams": 5,
        "early_stopping": True
    }
}

print(f"📝 Prompt: '{prompt}'")
print("-"*60)

for name, params in strategies.items():
    print(f"\n🎯 {name}:")
    output = generate_text(gen_model, tokenizer, prompt, max_length=80, **params)
    print(f"   {output}")

In [ ]:
# Visualize generation strategies
print("📊 DECODING STRATEGY COMPARISON")
print("="*60)
print("""
┌─────────────────────┬─────────────────┬─────────────────────────────────┐
│ Strategy            │ Parameters      │ Best For                        │
├─────────────────────┼─────────────────┼─────────────────────────────────┤
│ Greedy              │ -               │ Deterministic tasks             │
│ Temperature         │ temp=0.1-2.0    │ Controlling creativity          │
│ Top-K               │ k=10-100        │ Limiting vocabulary             │
│ Top-P (Nucleus)     │ p=0.9-0.95      │ Dynamic vocabulary cutoff       │
│ Beam Search         │ beams=3-10      │ Translation, quality focus      │
│ Contrastive         │ penalty=1.0     │ Reducing repetition             │
└─────────────────────┴─────────────────┴─────────────────────────────────┘

💡 Common combinations:
   - Chatbots: Top-P=0.9, Temperature=0.7
   - Creative writing: Temperature=0.9, Top-K=50
   - Code generation: Temperature=0.2, Top-P=0.95
   - Translation: Beam Search with 4-5 beams
""")

---

## 💾 Section 9: Saving, Loading & Deploying Models

In [ ]:
# Saving and Loading Models
print("💾 SAVING & LOADING MODELS")
print("="*60)

# Save LoRA adapter (small!)
lora_save_path = "./lora_adapter"
# model_lora.save_pretrained(lora_save_path)  # Uncomment to save
print(f"✅ LoRA adapter would be saved to: {lora_save_path}")
print("   Size: ~10-50 MB (vs GBs for full model)")

# Save tokenizer
# tokenizer.save_pretrained(lora_save_path)

# To load later:
print("""
📥 Loading a fine-tuned model:

from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "./lora_adapter")

# For inference, merge LoRA weights into base model
model = model.merge_and_unload()  # Optional: Makes inference faster
""")

# Merge and unload for deployment
print("\n🔧 Merging LoRA weights for deployment:")
print("   merged_model = model.merge_and_unload()")
print("   This combines LoRA weights with base model for faster inference.")

---

## 🚀 Section 10: Advanced Topics - RLHF & DPO

### Reinforcement Learning from Human Feedback (RLHF)

The technique that made ChatGPT so good at following instructions:

```
┌────────────────────────────────────────────────────────────────────────┐
│                           RLHF PIPELINE                                 │
├────────────────────────────────────────────────────────────────────────┤
│                                                                        │
│  1. PRE-TRAINING (Base LLM)                                            │
│     └─> Train on massive text data (next token prediction)             │
│                                                                        │
│  2. SUPERVISED FINE-TUNING (SFT)                                       │
│     └─> Train on instruction-response pairs                            │
│                                                                        │
│  3. REWARD MODEL TRAINING                                              │
│     ├─> Collect human comparisons: "Which response is better?"         │
│     └─> Train a model to predict human preferences                     │
│                                                                        │
│  4. REINFORCEMENT LEARNING (PPO)                                       │
│     └─> Optimize policy to maximize reward while staying close         │
│         to SFT model (KL divergence penalty)                           │
│                                                                        │
└────────────────────────────────────────────────────────────────────────┘
```

### Direct Preference Optimization (DPO)

A simpler alternative to RLHF - no reward model needed!

In [ ]:
# DPO Training Example (using trl library)
print("🎯 DIRECT PREFERENCE OPTIMIZATION (DPO)")
print("="*60)

# DPO loss function (simplified)
def dpo_loss(policy_chosen_logps, policy_rejected_logps, 
             ref_chosen_logps, ref_rejected_logps, beta=0.1):
    """
    DPO Loss = -log(σ(β * (log(π(y_w|x)/π_ref(y_w|x)) - log(π(y_l|x)/π_ref(y_l|x)))))
    
    Where:
    - y_w = chosen (winning) response
    - y_l = rejected (losing) response
    - π = policy model
    - π_ref = reference model (frozen)
    - β = temperature parameter
    """
    chosen_rewards = beta * (policy_chosen_logps - ref_chosen_logps)
    rejected_rewards = beta * (policy_rejected_logps - ref_rejected_logps)
    
    loss = -F.logsigmoid(chosen_rewards - rejected_rewards).mean()
    
    return loss

print("""
📊 DPO vs RLHF:
┌─────────────────────┬─────────────────────────────────────────────┐
│ Aspect              │ RLHF            │ DPO                       │
├─────────────────────┼─────────────────┼───────────────────────────┤
│ Reward Model        │ Required        │ Not needed                │
│ Complexity          │ High            │ Low                       │
│ Stability           │ Can be unstable │ More stable               │
│ Compute             │ Higher          │ Lower                     │
│ Performance         │ SOTA            │ Competitive               │
└─────────────────────┴─────────────────┴───────────────────────────┘

💡 DPO directly optimizes the policy using preference data,
   bypassing the need for a separate reward model!
""")

---

## 📋 Section 11: Complete Training Checklist

### Before Training:
- [ ] Choose base model appropriate for your task and compute budget
- [ ] Prepare high-quality training data (garbage in = garbage out!)
- [ ] Clean and deduplicate data
- [ ] Split data: train/val/test
- [ ] Set up proper prompt templates

### During Training:
- [ ] Monitor loss curves (train AND validation)
- [ ] Watch for overfitting (val loss increasing)
- [ ] Check GPU memory usage
- [ ] Log metrics with Weights & Biases or TensorBoard
- [ ] Save checkpoints regularly

### After Training:
- [ ] Evaluate on held-out test set
- [ ] Qualitative evaluation (look at actual outputs!)
- [ ] Compare with baseline model
- [ ] Test for biases and safety issues
- [ ] Save model and training config

In [ ]:
# Common hyperparameters and their typical ranges
print("🎛️ HYPERPARAMETER GUIDE")
print("="*60)

hyperparams = """
┌────────────────────────┬─────────────────┬─────────────────────────────────┐
│ Hyperparameter         │ Typical Range   │ Notes                           │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ Learning Rate          │ 1e-5 to 5e-4    │ Lower for full FT, higher for   │
│                        │                 │ LoRA. Start with 2e-5.          │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ Batch Size             │ 8 to 128        │ Larger = more stable, needs more│
│ (effective)            │                 │ memory. Use grad accumulation.  │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ Epochs                 │ 1 to 5          │ LLMs often need only 1-3 epochs │
│                        │                 │ Overfitting is easy!            │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ Warmup Ratio           │ 0.03 to 0.1     │ Gradual LR increase at start    │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ Weight Decay           │ 0.01 to 0.1     │ Regularization                  │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ LoRA Rank (r)          │ 8 to 64         │ Higher = more capacity, slower  │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ LoRA Alpha             │ 16 to 64        │ Usually 2x rank                 │
├────────────────────────┼─────────────────┼─────────────────────────────────┤
│ Max Sequence Length    │ 512 to 4096     │ Longer = more memory            │
└────────────────────────┴─────────────────┴─────────────────────────────────┘
"""
print(hyperparams)

print("💡 Pro Tips:")
print("   1. Start with default values and tune one at a time")
print("   2. Use learning rate finder to find optimal LR")
print("   3. If loss doesn't decrease: LR too low or data issues")
print("   4. If loss spikes: LR too high, reduce by 10x")

---

## 🎓 Summary: Complete LLM Training Pipeline

```
┌────────────────────────────────────────────────────────────────────────────┐
│                    COMPLETE LLM TRAINING PIPELINE                          │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  📥 1. DATA PREPARATION                                                    │
│     ├─> Collect instruction-response pairs                                 │
│     ├─> Format into prompt templates                                       │
│     ├─> Tokenize with appropriate max_length                               │
│     └─> Split: Train (90%) / Validation (10%)                              │
│                                                                            │
│  🤖 2. MODEL SETUP                                                         │
│     ├─> Choose base model (GPT-2, LLaMA, Mistral, etc.)                   │
│     ├─> Apply LoRA for efficient training                                  │
│     └─> Optional: Quantization for memory savings                          │
│                                                                            │
│  🏋️ 3. TRAINING                                                            │
│     ├─> Set hyperparameters (LR, batch size, epochs)                      │
│     ├─> Configure optimizer (AdamW) and scheduler (cosine)                │
│     ├─> Train with gradient accumulation                                   │
│     └─> Monitor loss and save checkpoints                                  │
│                                                                            │
│  📊 4. EVALUATION                                                          │
│     ├─> Calculate perplexity on validation set                            │
│     ├─> Generate sample outputs                                            │
│     └─> Compare with baseline                                              │
│                                                                            │
│  🚀 5. DEPLOYMENT                                                          │
│     ├─> Merge LoRA weights (optional)                                      │
│     ├─> Export model                                                       │
│     └─> Set up inference pipeline                                          │
│                                                                            │
└────────────────────────────────────────────────────────────────────────────┘
```

## 🚀 Next Steps

1. **Try with a larger model** - LLaMA 2 7B, Mistral 7B
2. **Experiment with different datasets** - OpenOrca, Dolphin, WizardLM
3. **Learn about DPO/RLHF** - Align model with human preferences
4. **Explore quantization** - GGUF, AWQ, GPTQ for deployment
5. **Deploy your model** - vLLM, TGI, Ollama

## 📚 Recommended Resources

- **Papers**: "Attention is All You Need", "LoRA", "QLoRA", "DPO"
- **Libraries**: Hugging Face, Unsloth (2x faster LoRA), Axolotl
- **Datasets**: HuggingFace Hub (tatsu-lab/alpaca, OpenOrca)
- **Compute**: Google Colab (free GPU), Lambda Labs, RunPod

---
*Built for aspiring LLM engineers 🚀*